# Reshaping Data

This notebook reshapes the output from the notebook `1_Methodology.ipynb` to generate the final output per level 2/3/4 boundary.

## Setup

Importing the relevant packages and loading all the relevant datasets.

The relevant datasets are as follows
1. `phl_pixels_all.csv`, which is generated by running `1_Methodology.ipynb`, as a dataframe.
2. [Admin boundaries](https://data.humdata.org/dataset/philippines-administrative-levels-0-to-3), to which we will join the percent completeness data
3. The previous output for each admin boundary level, to which we'll join our new output from this notebook

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import geopandas as gpd
import matplotlib.pyplot as plt
from scipy.stats.stats import pearsonr

import os
import wget

### Loading mapped and unmapped PH pixels

In [2]:
phl_pixels_all = pd.read_csv("../data/phl_pixels_all.csv")

In [3]:
phl_pixels_all.head()

,index,ADM3_PCODE,RURBAN,ADM4_PCODE_NAME,ADM2_PCODE,status
0,1,PH020902000,R,020902010_Santa Rosa (Kaynatuan),PH020900000,mapped
1,3,PH020902000,R,020902010_Santa Rosa (Kaynatuan),PH020900000,mapped
2,4,PH020902000,R,020902010_Santa Rosa (Kaynatuan),PH020900000,mapped
3,5,PH020902000,R,020902010_Santa Rosa (Kaynatuan),PH020900000,mapped
4,8,PH020902000,R,020902010_Santa Rosa (Kaynatuan),PH020900000,mapped


### Loading admin boundaries

In [4]:
phl_adm2 = gpd.read_file(
    "../download_data/phl_adm_all/phl_admbnda_adm2_psa_namria_20200529.shp"
)

In [5]:
phl_adm3 = gpd.read_file(
    "../download_data/phl_adm_all/phl_admbnda_adm3_psa_namria_20200529.shp"
)

In [6]:
phl_adm4 = gpd.read_file(
    "../download_data/phl_adm_2015_level4_barangay.gpkg/phl_adm_2015_level4_barangay.gpkg"
)

### Loading previous output

In [7]:
province_output = pd.read_csv('../download_data/mapthegap-phl-adm2-2021-05-29.csv')

In [8]:
citymuni_output = pd.read_csv('../download_data/mapthegap-phl-adm3-2021-05-29.csv')

In [9]:
brgy_output = pd.read_csv('../download_data/mapthegap-phl-adm4-2021-05-29.csv')

## Reshaping

In the following code snippets, we use a pivot table to get the number of mapped and unmapped pixels for each administrative region, then calculating the percent completeness

We repeat the same steps for ADM2 - ADM4, using `ADM2_PCODE`, `ADM3_PCODE`, and `ADM4_PCODE_NAME` as indices. 

### Level 2 Boundaries (provinces)

In [10]:
adm2_df = pd.pivot_table(phl_pixels_all[["ADM2_PCODE","status", "index"]], index = ["ADM2_PCODE"], columns = ["status"], aggfunc="count")

In [11]:
adm2_df.head()

index         
status      mapped unmapped
ADM2_PCODE                 
PH012800000  45646    18479
PH012900000  54459    23735
PH013300000  32348    42918
PH015500000  17768   238488
PH020900000    998      187

The resulting dataframe has a multiindex, which we will fix in the next code blocks

In [12]:
adm2_df.columns = adm2_df.columns.get_level_values(1)
adm2_df = adm2_df.reset_index().reset_index()

In [13]:
# Dataframe now has a regular index!
adm2_df.head()

status,index,ADM2_PCODE,mapped,unmapped
0,0,PH012800000,45646,18479
1,1,PH012900000,54459,23735
2,2,PH013300000,32348,42918
3,3,PH015500000,17768,238488
4,4,PH020900000,998,187


In [14]:
# Drop the index column 
adm2_df.drop(["index"],axis = 1,inplace=True)

# Renaming columns to fit previous convention
adm2_df.rename(columns = {
    'mapped':'pixels_withbuilding_june2021',
    'unmapped':'pixels_nobuilding_june2021'
    },
    inplace=True
)

# Adding a column for percent completeness
adm2_df['percentage_completeness_june2021'] = (adm2_df['pixels_withbuilding_june2021']/(adm2_df['pixels_withbuilding_june2021'] + adm2_df['pixels_nobuilding_june2021'])) * 100

In [15]:
adm2_df.head()

status,ADM2_PCODE,pixels_withbuilding_june2021,pixels_nobuilding_june2021,percentage_completeness_june2021
0,PH012800000,45646,18479,71.182846
1,PH012900000,54459,23735,69.646009
2,PH013300000,32348,42918,42.978237
3,PH015500000,17768,238488,6.933691
4,PH020900000,998,187,84.219409


We've successfully calculated the percentage completeness for level 2! Next, we will join the data with the admin boundaries information.

In [16]:
# Create new dataframe that will store phl_adm2 
# with percentage completness output
phl_adm2_with_output = phl_adm2

# Get only the columns we need to identify region
phl_adm2_with_output = phl_adm2_with_output[[
    'ADM2_EN',
    'ADM2_PCODE',
    'ADM2_REF',
    'ADM2ALT1EN',
    'ADM2ALT2EN'
 ]]

In [17]:
# Left joining percentage completness values 
# to their respective regions
phl_adm2_with_output = pd.merge(phl_adm2_with_output,adm2_df,how="left", on = "ADM2_PCODE")

In [18]:
phl_adm2_with_output.head()

,ADM2_EN,ADM2_PCODE,ADM2_REF,ADM2ALT1EN,ADM2ALT2EN,pixels_withbuilding_june2021,pixels_nobuilding_june2021,percentage_completeness_june2021
0,Abra,PH140100000,None,None,None,13862,5080,73.181290
1,Agusan del Norte,PH160200000,None,None,None,15586,24304,39.072449
2,Agusan del Sur,PH160300000,None,None,None,3549,37551,8.635036
3,Aklan,PH060400000,None,None,None,21777,24894,46.660667
4,Albay,PH050500000,None,None,None,32784,25916,55.850085


The last step will be to join it to the previous output and save it to file.

In [19]:
# Left joining the new percent completness values 
# to the existing values using ADM2_PCODE as index
# For the second dataframe, we only keep wanted columns
province_output_june2021 = pd.merge(
    province_output,
    phl_adm2_with_output[['ADM2_PCODE',\
                          'pixels_withbuilding_june2021',\
                          'pixels_nobuilding_june2021',\
                          'percentage_completeness_june2021'\
                         ]],
    how="left", 
    on = "ADM2_PCODE"
)

In [20]:
# Scroll through the columns to see how percentage completeness increased over time!
province_output_june2021.head()

,ADM2_EN,ADM2_PCODE,ADM2_REF,ADM2ALT1EN,ADM2ALT2EN,pixels_withbuilding_june2020,pixels_nobuilding_june2020,percentage_completeness_june2020,pixels_withbuilding_may2021,pixels_nobuilding_may2021,percentage_completeness_may2021,pixels_withbuilding_june2021,pixels_nobuilding_june2021,percentage_completeness_june2021
0,Abra,PH140100000,NaN,NaN,NaN,13790,5152,72.801183,13862,5080,73.181290,13862,5080,73.181290
1,Agusan del Norte,PH160200000,NaN,NaN,NaN,4202,35688,10.533968,15586,24304,39.072449,15586,24304,39.072449
2,Agusan del Sur,PH160300000,NaN,NaN,NaN,2953,38147,7.184915,3422,37678,8.326034,3549,37551,8.635036
3,Aklan,PH060400000,NaN,NaN,NaN,14693,31978,31.482077,21519,25152,46.107861,21777,24894,46.660667
4,Albay,PH050500000,NaN,NaN,NaN,27168,31532,46.282794,32756,25944,55.802385,32784,25916,55.850085


In [21]:
# Save to .csv file
filename = "../data/mapthegap-phl-adm2-2021-06-28.csv"
province_output_june2021.to_csv(filename)

Mostly same steps will be followed for the level 3 and 4 boundaries

### Level 3 (cities and municipalities)

In [22]:
# Pivot table of mapped and unmapped pixels
adm3_df = pd.pivot_table(phl_pixels_all[["ADM3_PCODE","status", "index"]], index = ["ADM3_PCODE"], columns = ["status"], aggfunc="count")

# Fix dataframe index
adm3_df.columns = adm3_df.columns.get_level_values(1)
adm3_df = adm3_df.reset_index().reset_index()

In [23]:
# Dataframe now has a regular index!
adm3_df.head()

status,index,ADM3_PCODE,mapped,unmapped
0,0,PH012801000,184,38
1,1,PH012802000,2965,883
2,2,PH012803000,2284,928
3,3,PH012804000,1573,334
4,4,PH012805000,3800,2193


In [24]:
# Drop the index column 
adm3_df.drop(["index"],axis = 1,inplace=True)

# Renaming columns to fit previous convention
adm3_df.rename(columns = {
    'mapped':'pixels_withbuilding_june2021',
    'unmapped':'pixels_nobuilding_june2021'
    },
    inplace=True
)

# Adding a column for percent completeness
adm3_df['percentage_completeness_june2021'] = (adm3_df['pixels_withbuilding_june2021']/(adm3_df['pixels_withbuilding_june2021'] + adm3_df['pixels_nobuilding_june2021'])) * 100

In [25]:
adm3_df.head()

status,ADM3_PCODE,pixels_withbuilding_june2021,pixels_nobuilding_june2021,percentage_completeness_june2021
0,PH012801000,184,38,82.882883
1,PH012802000,2965,883,77.053015
2,PH012803000,2284,928,71.108344
3,PH012804000,1573,334,82.485579
4,PH012805000,3800,2193,63.407309


In [26]:
# Create new dataframe that will store phl_adm3
# with percentage completness output
phl_adm3_with_output = phl_adm3

# Get only the columns we need to identify region
phl_adm3_with_output = phl_adm3_with_output[[
    'ADM3_EN',
    'ADM3_PCODE',
    'ADM3_REF',
    'ADM3ALT1EN',
    'ADM3ALT2EN'
 ]]

In [27]:
# Left joining percentage completness values 
# to their respective regions
phl_adm3_with_output = pd.merge(phl_adm3_with_output,adm3_df,how="left", on = "ADM3_PCODE")

In [28]:
phl_adm3_with_output.head()

,ADM3_EN,ADM3_PCODE,ADM3_REF,ADM3ALT1EN,ADM3ALT2EN,pixels_withbuilding_june2021,pixels_nobuilding_june2021,percentage_completeness_june2021
0,Aborlan,PH175301000,None,None,None,652.0,3541.0,15.549726
1,Abra de Ilog,PH175101000,None,None,None,1316.0,728.0,64.383562
2,Abucay,PH030801000,None,None,None,1993.0,649.0,75.435276
3,Abulug,PH021501000,None,None,None,3405.0,918.0,78.764747
4,Abuyog,PH083701000,None,None,None,1473.0,988.0,59.853718


The last step will be to join it to the previous output and save it to file.

In [29]:
# Left joining the new percent completness values 
# to the existing values using ADM3_PCODE as index
# For the second dataframe, we only keep wanted columns
citymuni_output_june2021 = pd.merge(
    citymuni_output,
    phl_adm3_with_output[['ADM3_PCODE',\
                          'pixels_withbuilding_june2021',\
                          'pixels_nobuilding_june2021',\
                          'percentage_completeness_june2021'\
                         ]],
    how="left", 
    on = "ADM3_PCODE"
)

In [30]:
# Scroll through the columns to see how percentage completeness increased over time!
citymuni_output_june2021.head()

,ADM3_EN,ADM3_PCODE,ADM3_REF,ADM3ALT1EN,ADM3ALT2EN,pixels_withbuilding_june2020,pixels_nobuilding_june2020,percentage_completeness_june2020,pixels_withbuilding_may2021,pixels_nobuilding_may2021,percentage_completeness_may2021,pixels_withbuilding_june2021,pixels_nobuilding_june2021,percentage_completeness_june2021
0,Aborlan,PH175301000,NaN,NaN,NaN,594,3599,14.166468,652,3541,15.549726,652.0,3541.0,15.549726
1,Abra de Ilog,PH175101000,NaN,NaN,NaN,1315,729,64.334638,1316,728,64.383562,1316.0,728.0,64.383562
2,Abucay,PH030801000,NaN,NaN,NaN,1996,646,75.548827,1993,649,75.435276,1993.0,649.0,75.435276
3,Abulug,PH021501000,NaN,NaN,NaN,3396,927,78.556558,3405,918,78.764747,3405.0,918.0,78.764747
4,Abuyog,PH083701000,NaN,NaN,NaN,1456,1005,59.162942,1473,988,59.853718,1473.0,988.0,59.853718


In [31]:
# Save to .csv file
filename = "../data/mapthegap-phl-adm3-2021-06-28.csv"
citymuni_output_june2021.to_csv(filename)

### Level 4 (barangays)

Note: Instead of using PCODE, we use `ADM4_PCODE_NAME` for this section

In [32]:
# Pivot table of mapped and unmapped pixels
adm4_df = pd.pivot_table(phl_pixels_all[["ADM4_PCODE_NAME","status", "index"]], index = ["ADM4_PCODE_NAME"], columns = ["status"], aggfunc="count")

# Fix dataframe index
adm4_df.columns = adm4_df.columns.get_level_values(1)
adm4_df = adm4_df.reset_index().reset_index()

In [33]:
# Dataframe now has a regular index!
adm4_df.head()

status,index,ADM4_PCODE_NAME,mapped,unmapped
0,0,012801001_Adams (Pob.),184.0,38.0
1,1,012802001_Bani,89.0,52.0
2,2,012802002_Buyon,143.0,54.0
3,3,012802003_Cabaruan,151.0,56.0
4,4,012802004_Cabulalaan,80.0,21.0


In [34]:
# Drop the index column 
adm4_df.drop(["index"],axis = 1,inplace=True)

# Renaming columns to fit previous convention
adm4_df.rename(columns = {
    'mapped':'pixels_withbuilding_june2021',
    'unmapped':'pixels_nobuilding_june2021'
    },
    inplace=True
)

# Adding a column for percent completeness
adm4_df['percentage_completeness_june2021'] = (adm4_df['pixels_withbuilding_june2021']/(adm4_df['pixels_withbuilding_june2021'] + adm4_df['pixels_nobuilding_june2021'])) * 100

In [35]:
adm4_df.head()

status,ADM4_PCODE_NAME,pixels_withbuilding_june2021,pixels_nobuilding_june2021,percentage_completeness_june2021
0,012801001_Adams (Pob.),184.0,38.0,82.882883
1,012802001_Bani,89.0,52.0,63.120567
2,012802002_Buyon,143.0,54.0,72.588832
3,012802003_Cabaruan,151.0,56.0,72.946860
4,012802004_Cabulalaan,80.0,21.0,79.207921


In [36]:
# Create new dataframe that will store phl_adm4
# with percentage completness output
phl_adm4_with_output = phl_adm4

# Get only the columns we need to identify region
phl_adm4_with_output = phl_adm4_with_output[[
    'Reg_Code',
    'Reg_Name',
    'Pro_Code',
    'Bgy_Code',
    'Bgy_Name',
    'RURBAN'
 ]]

# Create new column indicating both the pcode and name of the barangay
phl_adm4_with_output["ADM4_PCODE_NAME"] = phl_adm4_with_output["Bgy_Code"] + "_" + phl_adm4_with_output["Bgy_Name"]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [37]:
# Left joining percentage completness values 
# to their respective regions
phl_adm4_with_output = pd.merge(phl_adm4_with_output,adm4_df,how="left", on = "ADM4_PCODE_NAME")

In [38]:
phl_adm4_with_output.head()

,Reg_Code,Reg_Name,Pro_Code,Bgy_Code,Bgy_Name,RURBAN,ADM4_PCODE_NAME,pixels_withbuilding_june2021,pixels_nobuilding_june2021,percentage_completeness_june2021
0,110000000,REGION XI (DAVAO REGION),118200000,118206010,Sawangan,R,118206010_Sawangan,6.0,68.0,8.108108
1,180000000,NEGROS ISLAND REGION (NIR),184500000,184501048,Felisa,U,184501048_Felisa,3.0,271.0,1.094891
2,120000000,REGION XII (SOCCSKSARGEN),126300000,126311020,Simsiman,R,126311020_Simsiman,147.0,59.0,71.359223
3,150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),157000000,157001001,Balimbing Proper,R,157001001_Balimbing Proper,5.0,88.0,5.376344
4,150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),157000000,157001002,Batu-batu (Pob.),R,157001002_Batu-batu (Pob.),25.0,168.0,12.953368


The last step will be to join it to the previous output and save it to file.

In [39]:
# Left joining the new percent completness values 
# to the existing values using ADM3_PCODE as index
# For the second dataframe, we only keep wanted columns
brgy_output_june2021 = pd.merge(
    brgy_output,
    phl_adm4_with_output[['ADM4_PCODE_NAME',\
                          'pixels_withbuilding_june2021',\
                          'pixels_nobuilding_june2021',\
                          'percentage_completeness_june2021'\
                         ]],
    how="left", 
    on = "ADM4_PCODE_NAME"
)

In [40]:
# Scroll through the columns to see how percentage completeness increased over time!
brgy_output_june2021.head()

,Reg_Code,Reg_Name,Pro_Code,Pro_Name,Mun_Code,Mun_Name,Bgy_Code,Bgy_Name,RURBAN,ADM4_PCODE_NAME,pixels_withbuilding_june2020,pixels_nobuilding_june2020,percentage_completeness_june2020,ADM4_PCODE,pixels_withbuilding_may2021,pixels_nobuilding_may2021,percentage_completeness_may2021,pixels_withbuilding_june2021,pixels_nobuilding_june2021,percentage_completeness_june2021
0,110000000,REGION XI (DAVAO REGION),118200000,COMPOSTELA VALLEY,118206000.0,MAWAB,118206010.0,Sawangan,R,118206010_Sawangan,3,71,4.054054,118206010.0,6,68,8.108108,6.0,68.0,8.108108
1,180000000,NEGROS ISLAND REGION (NIR),184500000,NEGROS OCCIDENTAL,184501000.0,BACOLOD CITY (Capital),184501048.0,Felisa,U,184501048_Felisa,2,272,0.729927,184501048.0,3,271,1.094891,3.0,271.0,1.094891
2,120000000,REGION XII (SOCCSKSARGEN),126300000,SOUTH COTABATO,126311000.0,NORALA,126311020.0,Simsiman,R,126311020_Simsiman,116,90,56.310680,126311020.0,147,59,71.359223,147.0,59.0,71.359223
3,150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),157000000,TAWI-TAWI,157001000.0,PANGLIMA SUGALA (BALIMBING),157001001.0,Balimbing Proper,R,157001001_Balimbing Proper,0,93,0.000000,157001001.0,5,88,5.376344,5.0,88.0,5.376344
4,150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),157000000,TAWI-TAWI,157001000.0,PANGLIMA SUGALA (BALIMBING),157001002.0,Batu-batu (Pob.),R,157001002_Batu-batu (Pob.),11,182,5.699482,157001002.0,25,168,12.953368,25.0,168.0,12.953368


In [41]:
# Save to .csv file
filename = "../data/mapthegap-phl-adm4-2021-06-28.csv"
brgy_output_june2021.to_csv(filename)